In [43]:
#Use last cell for testing reaction networks

import numpy as np
from scipy.optimize import linprog

import itertools
from math import gcd
from functools import reduce

import math


In [44]:
#Following function takes a vector, and subtracts off a given stoichiometric vector 
#(also give the additional information of whether the reaction is reversible)
#This is the key step in producing new extreme vectors for the cone

def subtract_off(set_vector, stoich_vector, letter):
    # Extracting indices where stoich_vector is negative and positive
    J_1 = np.where(np.array(stoich_vector) < 0)[0]
    J_2 = np.where(np.array(stoich_vector) > 0)[0]

    # Initialize alphas
    alpha_1 = 0
    alpha_2 = 0

    if letter == 'I':
        # Calculating alpha_1 for J_1 indices to be negative
        alphas_for_J1_alpha1 = [max(0, -set_vector[i] / stoich_vector[i]) for i in J_1 if stoich_vector[i] != 0]
        if alphas_for_J1_alpha1:
            alpha_1 = max(alphas_for_J1_alpha1)

        # Calculating alpha_2 for J_1 indices to be positive
        alphas_for_J1_alpha2 = [max(0, set_vector[i] / stoich_vector[i]) for i in J_1 if stoich_vector[i] != 0]
        if alphas_for_J1_alpha2:
            alpha_2 = max(alphas_for_J1_alpha2)

    elif letter == 'R':
        # Calculating alpha_1 for J_1 indices to be negative and J_2 to be positive
        alphas_for_J1_alpha1 = [max(0, -set_vector[i] / stoich_vector[i]) for i in J_1 if stoich_vector[i] != 0]
        alphas_for_J2_alpha1 = [max(0, -set_vector[i] / stoich_vector[i]) for i in J_2 if stoich_vector[i] != 0]
        alphas_alpha1 = alphas_for_J1_alpha1 + alphas_for_J2_alpha1
        if alphas_alpha1:
            alpha_1 = max(alphas_alpha1)

        # Calculating alpha_2 for J_1 indices to be positive and J_2 to be negative
        alphas_for_J1_alpha2 = [max(0, set_vector[i] / stoich_vector[i]) for i in J_1 if stoich_vector[i] != 0]
        alphas_for_J2_alpha2 = [max(0, set_vector[i] / stoich_vector[i]) for i in J_2 if stoich_vector[i] != 0]
        alphas_alpha2 = alphas_for_J1_alpha2 + alphas_for_J2_alpha2
        if alphas_alpha2:
            alpha_2 = max(alphas_alpha2)

    new_vec_1 = np.array(set_vector) + alpha_1 * np.array(stoich_vector)
    new_vec_2 = np.array(set_vector) - alpha_2 * np.array(stoich_vector)

    return new_vec_1, new_vec_2




In [45]:
#The following takes all the cone vectors and all the reaction and their reversibilities
#and applies the subtract off procedure to everything, producing all possible new vectors form the starting
#vectors given as cone_vectors

def grow_cone_numpy(cone_vectors, stoich_matrix, reversibility):
    """
    Function to grow a cone using numpy arrays.
    """
    # Convert set of tuples to a numpy array, removing duplicates
    bigger_cone = np.array(list({tuple(row) for row in cone_vectors}))

    for cone_vector in cone_vectors:
        for stoich_vector, letter in zip(stoich_matrix, reversibility):
            # Applying subtract_off to each combination
            new_vec_1, new_vec_2 = subtract_off(cone_vector, stoich_vector, letter)

            # Adding the results to bigger_cone
            bigger_cone = np.vstack((bigger_cone, new_vec_1, new_vec_2))

    # Remove duplicates from the final result
    bigger_cone = np.unique(bigger_cone, axis=0)
    return bigger_cone








In [46]:
def is_in_positive_span(v, matrix):
    """
    Determine if vector v is in the positive span of the vectors in the matrix.
    """

    if matrix.size == 0 or matrix.shape[1] != len(v):
        # If the matrix is empty or its shape doesn't match, return False
        return False

    # Formulating the linear programming problem
    c = np.zeros(matrix.shape[0])  # Objective function (dummy)
    bounds = [(0, None) for _ in range(matrix.shape[0])]  # Non-negative coefficients

    # Solving the linear programming problem

    
    res = linprog(c, A_eq=matrix.T, b_eq=v, bounds=bounds, method='highs')

    return res.success  # Returns True if v is in the positive span of matrix

def find_minimal_spanning_set(cone_gen):
    """
    Produce a minimal set of vectors min_span from cone_gen.
    Finding a minimal set of vectors that generate the cone generated by cone_gen
    """
    min_span = cone_gen.copy()
    i = 0
    
    while i < len(min_span):
        v = min_span[i]
        others = np.delete(min_span, i, axis=0)
        
        if is_in_positive_span(v, others):
            # Remove the vector if it is in the positive span of the others
            min_span = others
        else:
            i += 1

    return min_span




In [47]:

#removes duplicate vectors
def remove_duplicates_numpy(vectors):
    """
    Remove duplicate vectors using only numpy arrays.
    """
    # Use numpy's unique function to remove duplicates
    unique_vectors = np.unique(vectors, axis=0)

    return unique_vectors



In [48]:

#removes duplicate vectors, new version
#meant to include some tolerance for duplicates

import numpy as np

def remove_duplicates_with_tolerance(vectors, epsilon=1e-6):
    """
    Removes duplicate vectors from the set of vectors based on a tolerance.

    Parameters:
    vectors (numpy.ndarray): An array of vectors.
    epsilon (float): The tolerance for considering vectors as duplicates.

    Returns:
    numpy.ndarray: The array of vectors with duplicates removed.
    """
    if vectors.size == 0:
        return vectors
    
    # Initialize an empty list to store unique vectors
    unique_vectors = [vectors[0]]

    for vector in vectors[1:]:
        # Check if the current vector is close to any of the unique vectors
        is_duplicate = np.any([np.all(np.abs(vector - uv) < epsilon) for uv in unique_vectors])
        if not is_duplicate:
            unique_vectors.append(vector)
    
    return np.array(unique_vectors)



In [49]:

#Takes a set of points ball_gen, and removes any points that are a convex combination of other points
#works by taking each point, checking if its in the convex combination of all the other points
#if it is not, it is added to a list of vectors that are not a convex combination of other points
#gives the set of points that are the vertices of the convex figure

def remove_convex_combinations(ball_gen):
    n = len(ball_gen)
    d = len(ball_gen[0])  # Assuming all vectors have the same dimension
    remaining_vectors = []

    for i in range(n):
        # Vector to test
        test_vector = ball_gen[i]

        # Other vectors
        other_vectors = np.concatenate([ball_gen[:i], ball_gen[i+1:]])

        # Objective: we don't need to minimize or maximize anything specific for this check
        c = np.zeros(other_vectors.shape[0])

        # Constraints: Ax = b, where A is the other vectors transposed, and b is the test vector
        A_eq = np.transpose(other_vectors)
        b_eq = test_vector

        # Bounds for each variable (weight) in the convex combination: they must be non-negative and sum to 1
        x_bounds = [(0, 1) for _ in range(other_vectors.shape[0])]
        A_sum = [np.ones(other_vectors.shape[0])]
        b_sum = [1]

        # Linear program
        res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, A_ub=A_sum, b_ub=b_sum, method='highs')

        # If the linear program is infeasible, then the test vector is not a convex combination of others
        if not res.success:
            remaining_vectors.append(test_vector)

    return np.array(remaining_vectors)




In [50]:

#Takes a set of points ball_gen, and removes any points that are a convex combination of other points
#VERSION 2

#works by checking if each point is a convex combination of other points, if it is, this point is removed


def remove_convex_combinations_v2(ball_gen):
    n = len(ball_gen)
    d = len(ball_gen[0])  # Assuming all vectors have the same dimension
    remaining_vectors = ball_gen

    for i in range(n):
        # Vector to test
        test_vector = ball_gen[i]

        # Other vectors
        other_vectors = np.concatenate([ball_gen[:i], ball_gen[i+1:]])

        # Objective: we don't need to minimize or maximize anything specific for this check
        c = np.zeros(other_vectors.shape[0])

        # Constraints: Ax = b, where A is the other vectors transposed, and b is the test vector
        A_eq = np.transpose(other_vectors)
        b_eq = test_vector

        # Bounds for each variable (weight) in the convex combination: they must be non-negative and sum to 1
        x_bounds = [(0, 1) for _ in range(other_vectors.shape[0])]
        A_sum = [np.ones(other_vectors.shape[0])]
        b_sum = [1]

        # Linear program
        res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, A_ub=A_sum, b_ub=b_sum, method='highs')

        # If the linear program is infeasible, then the test vector is not a convex combination of others
        if res.success:
            remaining_vectors = remaining_vectors[~np.all(remaining_vectors == test_vector, axis=1)]

    return np.array(remaining_vectors)



In [51]:

def check_positive_sum_to_zero(bigger_cone):
    """
    Check if there is a positive sum of vectors in bigger_cone that adds up to the zero vector.

    Parameters:
    bigger_cone (numpy.ndarray): A matrix whose columns are the vectors to be checked.

    Returns:
    bool: True if such a positive sum exists, False otherwise.
    """
    # Objective function (zeros because we only want feasibility)
    #print("check_positive_sum_to_zero,bigger_cone",bigger_cone)
    
    c = np.zeros(bigger_cone.shape[0])

    # Constraints: Ax = 0, where A is bigger_cone
    A_eq = bigger_cone.T
    b_eq = np.zeros(bigger_cone.shape[1])

    # Linear program to find non-negative weights that sum the vectors to the zero vector
    #note the 0.0001 is a little imprecise, just making sure not everything is 0
    res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=(0.0001, None), method='highs')
    #print("res",res)
    # If the problem is feasible, it means there is a positive sum that adds to zero
    return res.success




In [52]:
def extend_with_negatives(convex_set_points):
    """
    Takes a numpy array of vectors and appends the negative of each vector to the set.

    Parameters:
    convex_set_points (numpy.ndarray): An array of vectors.

    Returns:
    numpy.ndarray: The extended array with original and negative vectors.
    """
    # Calculate the negative of each vector
    negative_vectors = -convex_set_points

    # Append the negative vectors to the original array
    extended_convex_set_points = np.vstack((convex_set_points, negative_vectors))

    return extended_convex_set_points



In [53]:
#the following tests if a point is in the interior of a convex set, c is the point checked
#and the convex closure of points is the set, epsilon is a heuristic lower bound for the linear program
#used to solve this problem


def is_point_in_convex_hull(c, points, epsilon=1e-6):
    """
    Determines if the point c can be written as a convex combination of the points in the set.

    Parameters:
    c (numpy.ndarray): The target point.
    points (numpy.ndarray): An array of points where each row is a point.

    Returns:
    bool: True if the point c can be written as a convex combination of points, False otherwise.
    """
    


    n_points = points.shape[0]
    
    # Formulate the linear programming problem
    c_obj = np.zeros(n_points)  # Objective function: 0 since we just want to find feasibility

    # Constraints: points.T * lambda = c
    A_eq = np.vstack([points.T, np.ones(n_points)])
    b_eq = np.append(c, 1)

    # Bounds: 0 < lambda_i < 1
    bounds = [(epsilon, 1) for _ in range(n_points)]

    
    # Solve the linear programming problem
    

    result = linprog(c=c_obj, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

    

    
    
    return result.success



In [54]:
#takes the reaction vectors, their reversibility, and a number n that says how many times to run the procedure
#of producing new points for the convex norm ball
#Keeps doing the subtract off procedure and growing the norm ball, stops if reach a set that is closed
#Also appends the negatives of teh vectors to the norm ball, since if we are nonexpansive we 
#should have the negative vectors in the ball without an issue

def grow_convex_set(stoich_matrix, reversibility, n):
    global convex_success
    convex_success = 0
    
    initial_vector = np.array([stoich_matrix[0]])  # store the the first vector
    convex_set = np.array([stoich_matrix[0]])  # Initialize with the first vector
    global convex_state
    convex_state = 0
    
    global start_vector_interior
    start_vector_interior = 0
    
    for _ in range(n):
        # Grow the convex set
        bigger_convex_set = grow_cone_numpy(convex_set, stoich_matrix, reversibility)
        
        # Remove duplicates

        bigger_convex_set = remove_duplicates_numpy(bigger_convex_set)
        
        bigger_convex_set = remove_duplicates_with_tolerance(bigger_convex_set)

        
        # Remove vectors that are convex combinations of others
        #bigger_convex_set = remove_convex_combinations(bigger_convex_set)
        

        bigger_convex_set = remove_convex_combinations_v2(bigger_convex_set)

        
        #appends the negatives of the vectors to the set
        bigger_convex_set = extend_with_negatives(bigger_convex_set)
        
        
        if is_point_in_convex_hull(initial_vector, bigger_convex_set):
            #print("CONVEX SET FAILURE")
            start_vector_interior = 1
            return bigger_convex_set
        
        # Check if the set has changed
        if np.array_equal(bigger_convex_set, convex_set):
            #print("CONVEX SET SUCCESS")
            convex_state = 1
            convex_success = 1
            return bigger_convex_set
        else:
            convex_set = bigger_convex_set

    return convex_set





In [55]:
global convex_success

###NOTE USE VALUE n BELOW TO GROW THE CONVEX SET, this value can vary

#removes reaction vectors until we have a non-contractive system such that if we remove any vector we get a 
#contractive system

def process_arrays(stoich_matrix, reversibility):
    m = len(stoich_matrix)
    indices = np.arange(m)
    entries_removed = True
    
    reversibility = np.array(reversibility)
    
    while entries_removed:
        entries_removed = False
        m = len(stoich_matrix)
        indices = np.arange(m)
        for i in range(len(indices)):
            # Create reduced versions of stoich_matrix and reversibility
            reduced_indices = np.delete(indices, i)

            reduced_stoich_matrix = stoich_matrix[reduced_indices]
            
            reduced_reversibility = reversibility[reduced_indices]

            # Run the function on the reduced arrays
            grow_convex_set(reduced_stoich_matrix, reduced_reversibility, n=20)

            # Check the result and decide whether to keep the removal or not
            if convex_success == 0:
                # Remove the entry permanently
                stoich_matrix = reduced_stoich_matrix
                reversibility = reduced_reversibility
                indices = reduced_indices
                entries_removed = True
                break  # Exit the loop to restart the process after removal

    return stoich_matrix, reversibility



In [56]:

#Takes an array of signs corresponding to an orthant, produces vectors corresponding to which directions the 
#rates of the corresponding reactions could move
#Output is to be fed to checking if there is a positive sum of the vectors that adds to 0, so that we can
#argue that any cone will contain a starting vector in this region

#outputs a cone of vectors, for telling which orthant is a mixed region

def process_vectors(stoich_matrix, reversibility, signs):
    directions = []

    for i, vector in enumerate(stoich_matrix):
        if reversibility[i] == 'R':
            temp_vector = vector
        elif reversibility[i] == 'I':
            temp_vector = np.where(vector > 0, 0, vector)
            

        sign_matches = temp_vector * signs

        
        if any(sign_matches > 0):
            directions.append(vector)
        if any(sign_matches < 0):
            directions.append(-vector)

    return np.array(directions)



In [57]:
import numpy as np
from scipy.linalg import null_space

def orthogonal_complement_basis(row_vectors):
    """
    Finds the basis of the space orthogonal to the given row vectors.
    
    Parameters:
    row_vectors (np.ndarray): A 2D numpy array where each row is a vector.
    
    Returns:
    np.ndarray: A 2D numpy array where each row is a vector in the orthogonal complement space.
    """
    # Compute the null space of the matrix formed by the row vectors
    orthogonal_complement = null_space(row_vectors)
    
    # Transpose to get the row vectors as rows of the output matrix
    orthogonal_complement = orthogonal_complement.T
    
    return orthogonal_complement



In [58]:
import numpy as np
from scipy.optimize import linprog

#Chceks an orthant region, and whether it nontrivially intersects a given subspace

def find_vector(signs, stoich_matrix):
    #num_vectors, num_variables = stoich_matrix.shape
    A_eq = orthogonal_complement_basis(stoich_matrix)
    num_vectors, num_variables = A_eq.T.shape
    # Create the constraints for linprog
    #A_eq = stoich_matrix.T
    b_eq = np.zeros(num_variables)

    # Create the bounds based on the signs vector
    bounds = []
    for sign in signs:
        if sign < 0:
            bounds.append((None, -1e-6))  # Allow negative values only
        elif sign > 0:
            bounds.append((1e-6, None))   # Allow positive values only
        else:
            bounds.append((0, 0))         # Fix to zero

    # Objective function is not important since we're only interested in feasibility
    c = np.zeros(num_vectors)

    # Solve the linear programming problem
    result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

    if result.success:
        return result.x
    else:
        return(False)
        #raise ValueError("No solution found that satisfies the constraints.")




In [59]:
import numpy as np

#the following removes vectors from the stoich matrix until its rank reduces by 1

def reduce_matrix_to_rank(stoich_matrix, reversibility):
    original_rank = np.linalg.matrix_rank(stoich_matrix)
    reduced_matrix = np.copy(stoich_matrix)
    reduced_reversibility = np.copy(reversibility)
    
    for i in range(len(stoich_matrix)):
        reduced_matrix = np.delete(reduced_matrix, 0, axis=0)
        reduced_reversibility = np.delete(reduced_reversibility, 0)
        if np.linalg.matrix_rank(reduced_matrix) == original_rank - 1:
            return reduced_matrix, reduced_reversibility
    
    return reduced_matrix, reduced_reversibility



In [60]:
#turn below into function, take reaction network find startign vector
import numbers
import decimal

stoich_matrix = np.array([[1,0,0,0],[0, 0, -1, 0], [-1, 1, 0, 1], [0, -1, 1, 0], [0, 0, -1, 1]])
reversibility = ['R','R', 'R', 'R', 'R']


#the following generate vectors that represents each possible orhant/combination of signs we can have in R^n
def generate_vectors(length):
    # Generate all possible vectors of the given length with elements -1, 0, and 1
    vectors = list(itertools.product([-1, 0, 1], repeat=length))
    return vectors



#Finds a starting vector by checking each orthant, if it satisfies a certain linear equation, and if it 
#nontrivially intersects the stoichiometric subspace (reduced subspace)
def find_starting_vector(stoich_matrix, reversibility):
    reduced_stoich, reduced_reversibility = process_arrays(stoich_matrix, reversibility)
    reduced_rank_stoich, reduced_rank_reversibility = reduce_matrix_to_rank(reduced_stoich, reduced_reversibility)
    
    vector_length = stoich_matrix.shape[1]
    all_vectors = generate_vectors(vector_length)
    for signs in all_vectors:
        
        signs = np.array(signs)
        directions = process_vectors(reduced_rank_stoich, reduced_rank_reversibility, signs)
        

        
        if len(directions) == 0:

            intersect = find_vector(signs, reduced_stoich) #check if the signs orthant nontrivially intersects reduced_stoic

            if not isinstance(intersect, bool):
                return(intersect)
            else:
                continue
        

        
        if check_positive_sum_to_zero(directions) == True: #checks if orthant is mixed region

            intersect = find_vector(signs, reduced_stoich) #check if the signs orthant nontrivially intersects reduced_stoic

            if not isinstance(intersect, bool):
                return(intersect)


#normalizes a vector with respect to the l infinity norm
def normalize_vector_l_infinity(vector):
    # Calculate the L-infinity norm (maximum absolute value)
    max_value = np.max(np.abs(vector))
    
    # Avoid division by zero
    if max_value == 0:
        return vector
    
    # Normalize the vector
    normalized_vector = vector / max_value
    return normalized_vector            
            
            

In [61]:
def check_reaction_vector_in_cone_not_extreme(stoich_matrix, bigger_cone):
    for vector in stoich_matrix:
        if is_in_positive_span(vector, bigger_cone) or is_in_positive_span(-vector, bigger_cone):
            if not (is_positive_multiple(vector, bigger_cone) or is_positive_multiple(-vector, bigger_cone)):
                return True
    return False

In [62]:


def lcm(a, b):
    if a == 0 or b == 0:
        return max(a, b)
    return abs(a * b) // math.gcd(a, b)


def lcm_array(arr):
    # Flatten the array to 1D
    flat_arr = arr.flatten()
    
    # Filter out zeros
    flat_arr = flat_arr[flat_arr != 0]
    
    if len(flat_arr) == 0:
        return 0
    
    # Calculate the LCM of all elements in the flattened array
    lcm_result = flat_arr[0]
    for num in flat_arr[1:]:
        lcm_result = lcm(lcm_result, num)
    
    return lcm_result

# Example usage
arr = np.array([[0,2,5],[4,2,5]])
result = lcm_array(arr)
print(f"The least common multiple of the array is: {result}")

def gcd_array(arr):
    gcd_result = arr[0]
    for num in arr[1:]:
        gcd_result = math.gcd(gcd_result, num)
    return gcd_result

The least common multiple of the array is: 20


In [63]:
def are_matrices_close(matrix1, matrix2, tolerance=1e-2):
    # Check if the shapes of the matrices are the same
    if matrix1.shape != matrix2.shape:
        return False
    
    # Calculate the absolute differences
    difference = np.abs(matrix1 - matrix2)
    
    # Check if all differences are within the tolerance
    return np.all(difference <= tolerance)

In [64]:

#takes the bigger_cone matrix of extremes of the cone, and turns them into integer extreme vectors
#needs to take numpy arrays
def clean_up_cone_vectors(stoich_matrix, bigger_cone):
    stoich_matrx_lcm = lcm_array(stoich_matrix)
    int_matrix = bigger_cone * stoich_matrx_lcm * stoich_matrx_lcm

    rounded_int_matrix = np.round(int_matrix).astype(int)
    
    
    #check if the rounded matrix is very close to the original
    #result = are_matrices_close(int_matrix, rounded_int_matrix)

    result_matrix = np.array([
    vec // gcd_array(vec) for vec in rounded_int_matrix
    ])
    return result_matrix


    


In [65]:
def min_nonzero_abs_value(arr):
    # Filter out zero values and find the minimum nonzero absolute value
    nonzero_elements = arr[np.abs(arr) > 0]
    min_value = np.min(np.abs(nonzero_elements))
    return min_value

def normalize_vector(vector):
    # Find the minimum nonzero absolute value
    min_value = min_nonzero_abs_value(vector)
    
    # Divide the vector by this value
    normalized_vector = vector / min_value
    
    return normalized_vector



In [66]:


def max_absolute_value_rounded(array):
    """
    Find the maximum absolute value in a numpy array of unknown dimension and round it up.

    Parameters:
    array (numpy.ndarray): Input array.

    Returns:
    float: Maximum absolute value in the array, rounded up.
    """
    max_abs_value = np.max(np.abs(array))
    return int(np.ceil(max_abs_value))



In [67]:

def adjust_vector_to_integer(vector, max_value, epsilon):
    """
    Adjust a vector such that each entry is multiplied by an integer (at most max_value)
    to be within epsilon tolerance of an integer value, and then round it to the nearest integer.
    
    Parameters:
    vector (numpy.ndarray): Input vector.
    max_value (int): Maximum integer multiplier.
    epsilon (float): Tolerance for adjusting to integer value.

    Returns:
    numpy.ndarray: Adjusted vector.
    """
    adjusted_vector = vector.copy()
    
    #this is already done in different function
    ####added to normalize the vector first before trying for integers
    #first_nonzero = vector[np.nonzero(adjusted_vector)[0][0]]
    #adjusted_vector = adjusted_vector / np.abs(first_nonzero)
    #####
    n = len(adjusted_vector)
    
    for i in range(n):
        for k in range(1, max_value + 1):
            multiplied_value = adjusted_vector[i] * k
            if np.abs(multiplied_value - round(multiplied_value)) <= epsilon:
                adjusted_vector *= k
                adjusted_vector[i] = round(multiplied_value)
                break
    
    return adjusted_vector




In [68]:
def adjust_matrix_to_integers(matrix, max_value, epsilon= 0.01):
    """
    Apply the adjust_vector_to_integer function to each vector (row) in a 2D numpy array.

    Parameters:
    matrix (numpy.ndarray): Input 2D array.
    max_value (int): Maximum integer multiplier.
    epsilon (float): Tolerance for adjusting to integer value.

    Returns:
    numpy.ndarray: Adjusted 2D array.
    """
    adjusted_matrix = matrix.copy()
    
    for i in range(adjusted_matrix.shape[0]):
        adjusted_matrix[i] = adjust_vector_to_integer(adjusted_matrix[i], max_value, epsilon)
    
    return adjusted_matrix



In [69]:

def normalize_vectors_divide_by_nonzero_entry(matrix):
    """
    Normalize each vector in a 2D numpy array by dividing each vector by one of its nonzero entries.

    Parameters:
    matrix (numpy.ndarray): Input 2D array where each row is a vector.

    Returns:
    numpy.ndarray: Normalized 2D array.
    """
    normalized_matrix = matrix.copy()
    
    #need to treat things as floats for division
    normalized_matrix = normalized_matrix.astype(float)
    
    for i in range(normalized_matrix.shape[0]):
        # Find the first nonzero entry in the vector
        nonzero_entries = normalized_matrix[i][normalized_matrix[i] != 0]
        if len(nonzero_entries) > 0:
            first_nonzero = nonzero_entries[0]

            normalized_matrix[i] /= abs(first_nonzero)
    
    return normalized_matrix



In [70]:
import numpy as np

def convert_array_if_within_epsilon(array, epsilon = 0.01):
    """
    Convert elements in a numpy array to integers if all elements are within epsilon of an integer value.

    Parameters:
    array (numpy.ndarray): Input array.
    epsilon (float): Tolerance for converting to integer value.

    Returns:
    numpy.ndarray: Array with elements converted to integers if condition is met, otherwise the original array.
    """
    def is_within_epsilon(x, epsilon):
        return np.abs(x - round(x)) < epsilon
    
    if np.all(np.abs(array - np.round(array)) < epsilon):
        return np.round(array).astype(int)
    else:
        return array





In [71]:
import numpy as np

def process_vectors_zero_out_small(matrix, epsilon = 0.01):
    """
    Process each vector in a 2D numpy array by finding the largest entry,
    dividing the vector by this entry, zeroing entries with absolute value less than epsilon,
    and then multiplying back the largest entry.

    Parameters:
    matrix (numpy.ndarray): Input 2D array where each row is a vector.
    epsilon (float): Tolerance for zeroing small entries.

    Returns:
    numpy.ndarray: Processed 2D array.
    """
    processed_matrix = matrix.copy()
    
    for i in range(processed_matrix.shape[0]):
        vector = processed_matrix[i]
        # Find the largest entry in the vector
        largest_entry = np.max(np.abs(vector))
        
        if largest_entry == 0:
            continue  # Skip processing if the vector is all zeros
        
        # Divide the vector by the largest entry
        normalized_vector = vector / largest_entry
        
        # Zero entries with absolute value less than epsilon
        normalized_vector[np.abs(normalized_vector) < epsilon] = 0
        
        # Multiply back the largest entry
        processed_matrix[i] = normalized_vector * largest_entry
    
    return processed_matrix



In [72]:
#just applies multiple other functions all together

def matrix_cleaning(bigger_cone,max_value):
    
        bigger_cone = normalize_vectors_divide_by_nonzero_entry(bigger_cone)
        bigger_cone = adjust_matrix_to_integers(bigger_cone, max_value)
        bigger_cone = convert_array_if_within_epsilon(bigger_cone)
        bigger_cone = process_vectors_zero_out_small(bigger_cone)
        bigger_cone = remove_duplicates_with_tolerance(bigger_cone)
        
        

        bigger_cone = normalize_vectors_divide_by_nonzero_entry(bigger_cone)
        bigger_cone = adjust_matrix_to_integers(bigger_cone, max_value)
        bigger_cone = convert_array_if_within_epsilon(bigger_cone)
        bigger_cone = process_vectors_zero_out_small(bigger_cone)
        bigger_cone = remove_duplicates_with_tolerance(bigger_cone)
        
        return bigger_cone
    

In [73]:
#grows a vector, takse as the starting vector find_starting_vector(stoich_matrix, reversibility)
#important function that starts the forming of the cone
def grow_reduce_cone_new_starting(stoich_matrix, reversibility, n):
    """
    Function that iteratively grows and reduces a set of vectors.
    """
    global cone_state
    cone_state = 0
    
    vector = find_starting_vector(stoich_matrix, reversibility)
    

    
    
    vector = normalize_vector_l_infinity(vector)

    
    #IF WANT TO SEE WHAT VECTOR IS PICKED FOR START, PROCESS IS STEP 1, CAN BE GOOD FOR DEBUGGING
    print("starting vector",vector)
    
    cone_vectors = np.array([vector])
    
    
    for _ in range(n):
        # Grow the cone

        bigger_cone = grow_cone_numpy(cone_vectors, stoich_matrix, reversibility)

        # Remove duplicates
        bigger_cone = remove_duplicates_numpy(bigger_cone)
        # remove deuplicates once more with 10^-6 tolerance
        bigger_cone = remove_duplicates_with_tolerance(bigger_cone)
        # Find minimal spanning set
        
        
        if check_positive_sum_to_zero(bigger_cone) == True:
            #SUPRESSSING FOLLOWING PRINT, CAN BE USEFUL TO USE THOUGH TO SEE THAT CONE PROVABLY NOT EXISTS
            print("CONE FAILURE: CONE IS NOT POINTED")
            return cone_vectors
            #break
   
        cone_vectors_new = find_minimal_spanning_set(bigger_cone)

        if len(cone_vectors_new) == 0:
            break
        rank_cone_vectors_new = np.linalg.matrix_rank(cone_vectors_new)
        rank_bigger_cone = np.linalg.matrix_rank(bigger_cone)
        

        if check_reaction_vector_in_cone_not_extreme(stoich_matrix, cone_vectors_new):
            
            print("CONE FAILURE: REACTION VECTOR INSIDE CONE AND NOT EXTREMAL")
            return cone_vectors

        if rank_cone_vectors_new < rank_bigger_cone: #meant to check if we remove a dimension of the cone, it is not pointed, thus for strongly connected we should have no cone

            print("CONE FAILURE: CONE DIMENSION DECREASED (NOT POINTED)")
            return cone_vectors
            #break


        # Check if new set is the same as the old one
        if np.array_equal(cone_vectors_new, cone_vectors):
            print("CONE SUCCESS")
            #cleans the matrix by normalizing, rounding to integers,etc
            cone_vectors = matrix_cleaning(cone_vectors,max_absolute_value_rounded(stoich_matrix))
            cone_state = 1
            return cone_vectors
        else:
            cone_vectors = cone_vectors_new
    
    
    #####DOING A FINAL THING HERE

    
    bigger_cone = matrix_cleaning(bigger_cone,max_absolute_value_rounded(stoich_matrix))
    bigger_cone = find_minimal_spanning_set(bigger_cone)
    cone_vectors = bigger_cone
    
    #repeat things one more time as above
    
    bigger_cone = grow_cone_numpy(cone_vectors, stoich_matrix, reversibility)
    bigger_cone = find_minimal_spanning_set(bigger_cone)
    bigger_cone = matrix_cleaning(bigger_cone,max_absolute_value_rounded(stoich_matrix))
    cone_vectors_new = find_minimal_spanning_set(bigger_cone)

    
    if np.array_equal(cone_vectors_new, cone_vectors):
        print("CONE SUCCESS ON LAST TRY")
        cone_state = 1
        return cone_vectors
    
    ################
    
    #print("CONE PROCEDURE DID NOT TERMINATE \n")
    print("CONE PROCEDURE DID NOT TERMINATE")
    print("cone_vectors_new\n,",cone_vectors_new)
    print("cone_vectors\n",cone_vectors)
    
    print("stoich_matrix,",stoich_matrix, "\n")
    return cone_vectors




In [74]:
import numpy as np
#checks if a postive multiple of two vectors are within an epsilon of each other
def compare_vectors(vector1, vector2, epsilon=1e-2):
    vector1 = np.array(vector1, dtype=float)
    vector2 = np.array(vector2, dtype=float)
    
    # Check if any element in either vector is zero
    if np.all(vector1 == 0) or np.all(vector2 == 0):
        return False

    # Find an index i where both vectors are nonzero
    nonzero_indices = np.where((vector1 != 0) & (vector2 != 0))[0]
    if len(nonzero_indices) == 0:
        return False

    i = nonzero_indices[0]

    # Calculate the ratio d
    d = vector1[i] / vector2[i]

    
    # Check if d is less than zero
    if d < 0:
        return False

    # Calculate the difference vector
    difference = vector1 - d * vector2
    
    # Check if the absolute value of difference is less than epsilon
    if np.all(np.abs(difference) < epsilon):
        return True
    else:
        return False

    
 

In [75]:

def is_positive_multiple(target_vector, array_of_vectors, epsilon=1e-2):
    target_vector = np.array(target_vector, dtype=float)
    
    for vector in array_of_vectors:
        vector = np.array(vector, dtype=float)
        
        if compare_vectors(target_vector, vector, epsilon):
            return True
    
    return False



In [76]:
#checks for multiplicity, taking in account reversibility
#so we know if some vectors are positive or negative multiples of each other, taking into account reversibility, for
#skipping identical reactions when cycling through them to check

def is_multiple(target_vector, array_of_vectors, reversibility):
    target_vector = np.array(target_vector, dtype=float)
    
    for vector, rev in zip(array_of_vectors, reversibility):
        vector = np.array(vector, dtype=float)
        
        # Check if lengths are compatible
        if len(vector) != len(target_vector):
            continue
        
        # Calculate the ratios where vector elements are not zero
        with np.errstate(divide='ignore', invalid='ignore'):
            ratios = np.divide(target_vector, vector, out=np.full_like(target_vector, np.nan), where=vector != 0)
        
        # Remove NaNs from the ratios array
        valid_ratios = ratios[~np.isnan(ratios)]
        
        if rev == 'I':
            # Check if all valid ratios are the same and positive
            if len(valid_ratios) > 0 and np.all(valid_ratios > 0) and np.all(valid_ratios == valid_ratios[0]):
                return True
        elif rev == 'R':
            # Check if all valid ratios are the same and nonzero
            if len(valid_ratios) > 0 and np.all(valid_ratios != 0) and np.all(valid_ratios == valid_ratios[0]):
                return True
    
    return False



In [77]:

def generate_vectors_2(length,entries):
    """
    Generate all possible vectors of the given length with elements from entries.
    """
    return list(itertools.product(entries, repeat=length))



def enumerate_and_run(stoich_matrix, reversibility, n, entries):
    """
    Enumerate every length m vector with entries from 'entries', append it to stoich_matrix,
    and run grow_reduce_cone_new_starting.
    """
    m = stoich_matrix.shape[1]  # Length of one vector in stoich_matrix
    vectors = generate_vectors_2(m,entries)  # Generate all possible vectors

    for vector in vectors:
        # Convert the tuple to a numpy array and append it to stoich_matrix
        new_vector = np.array(vector)
        new_stoich_matrix = np.vstack([stoich_matrix, new_vector])
        
            
        if reversibility[-1] == 'I' and all(x >= 0 for x in new_vector):
            continue  # Skip the current iteration
        elif reversibility[-1] == 'I' and is_multiple(new_vector, stoich_matrix, reversibility):
            continue
        elif reversibility[-1] == 'R' and (is_multiple(new_vector, stoich_matrix, reversibility) or is_multiple(new_vector, stoich_matrix, reversibility)):
            continue
        

        
        result = grow_reduce_cone_new_starting(new_stoich_matrix, reversibility, n)
        global cone_state
        if cone_state == 1:
            #convex_set = grow_convex_set(stoich_matrix, reversibility, n)
            #print("convex_set \n",convex_set)
            
            print("new_stoich_matrix \n",new_stoich_matrix)
            print("reversibility", reversibility)
            print("cone \n",result)
            print("\n")
            

In [78]:

#for cycling through possible additional vectors and finding larger monotone systems, make sure the last entry in reversibiilty
#decribes if the added reaction is reversible or not, should have one more reversible entry than number of reaction vectors given
#n is the number of time we grow the cone/convex set, entries if the possible entries of the new appended vector
#sometimes it might not terminate if n is not the right size (usually 30 - 300), i.e., did not get close enough to the final cone



n = 50 #this n dictates how many time the program tries to grow the cone before terminating
#entries = np.array([-3,-2,-1,0,1,2,3])
entries = np.array([-1,0,1])




stoich_matrix = np.array([[-2, 1, 1], [1,0,-1],[-1,1,0]])
reversibility = ['R','R','R','I']


enumerate_and_run(stoich_matrix, reversibility, n, entries)



starting vector [-1. -1.  1.]
CONE FAILURE: REACTION VECTOR INSIDE CONE AND NOT EXTREMAL
starting vector [-1.   0.5  0.5]
CONE FAILURE: REACTION VECTOR INSIDE CONE AND NOT EXTREMAL
starting vector [-1.   0.5  0.5]
CONE FAILURE: REACTION VECTOR INSIDE CONE AND NOT EXTREMAL
starting vector [-0.5 -0.5  1. ]
CONE FAILURE: REACTION VECTOR INSIDE CONE AND NOT EXTREMAL
starting vector [-0.5 -0.5  1. ]
CONE FAILURE: REACTION VECTOR INSIDE CONE AND NOT EXTREMAL
starting vector [-0.5 -0.5  1. ]
CONE FAILURE: REACTION VECTOR INSIDE CONE AND NOT EXTREMAL
starting vector [-0.5 -0.5  1. ]
CONE FAILURE: REACTION VECTOR INSIDE CONE AND NOT EXTREMAL
starting vector [-0.5 -0.5  1. ]
CONE FAILURE: REACTION VECTOR INSIDE CONE AND NOT EXTREMAL
starting vector [-0.5 -0.5  1. ]
CONE FAILURE: REACTION VECTOR INSIDE CONE AND NOT EXTREMAL


In [79]:
#IF INPUT ALL 0 STOICH MATRIX, THROWS ERROR
#Below simply tests a given reaction network for monotonicity and non-expansivity
#simply give reaction vectors as row vectors in a numpy array
#and reversibility in another array, with 'R' for reversible, and 'I' for irreversible for the corresponding reaction row vector


n = 50 
#this number dictates how many times the program tries to produce new points for a cone/norm ball
#sometimes the program is converging to an object, but does not quite reach it, when this happens it takes longer to run
#may need to play with n in this case, so that it gets close enough to the real object to guess the object (i.e., look for nearby points with rational coordinates)




#example 1 in paper
stoich_matrix = np.array([[-13, 11, 7], [7,-9,0],[-2,1,2]])
reversibility = ['R','I','I']

#following is monotone for usual orthant
stoich_matrix = np.array([[-1, 1,0], [-2,0,1],[0,-1,1],[-3,1,1]])
reversibility = ['R','R','R','I']

#additional example, gives cone
stoich_matrix = np.array([[-1, -1, 1,0], [-2,1,0,1],[0,0,-1,1],[-3,2,1,1]])
reversibility = ['I','I','R','I']

#additional example, gives cone
stoich_matrix = np.array([[-1,1,1,0],[2,-1,0,-1],[1,0,-2,0],[0,0,0,-1]])
reversibility = ['R','I','I','R']

#example 3 in paper
stoich_matrix = np.array([[-1, -1, 1],[-1,0,1],[0,-1,1],[1,-1,0],[1,0,0],[0,1,0]])
reversibility = ['R','I','I','R','R','R']

#example 8.6 from the paper 'Monotonicity in chemical reaction systems'
stoich_matrix = np.array([[-1, -1, 2],[-1,0,1],[0,-1,1]])
reversibility = ['R','R','R']

#example 8.6 from the paper 'Monotonicity in chemical reaction systems'
stoich_matrix = np.array([[-1, -1, 2],[-1,0,1],[0,-1,1]])
reversibility = ['R','R','R']

#example 8.7 from the paper 'Monotonicity in chemical reaction systems', should not have a valid cone
stoich_matrix = np.array([[-1, -1, 1],[-1,1,0],[-2,0,1]])
reversibility = ['R','R','R']

#example 2 in paper
stoich_matrix = np.array([[-1,1,1,0],[1,-1,0,1],[1,0,-1,1],[1,-1,-1,-2]])
reversibility = ['I','I','I','I']

#example 4 in paper
stoich_matrix = np.array([[-1,-1,1],[-2,0,1],[-2,1,0],[0,2,-1],[2,-2,1],[1,-2,1],[-1,0,0],[0,-2,1],[0,-1,1]])
reversibility = ['R','I','I','R','I','I','R','R','I']




result = grow_reduce_cone_new_starting(stoich_matrix, reversibility, n)
print("cone \n",result)

convex_set = grow_convex_set(stoich_matrix, reversibility, n)

global convex_success
global start_vector_interior

if convex_success == 1:
    print("CONTRACTIVE SUCCESS")
    #print("convex_set \n",convex_set)


if start_vector_interior == 1:
    print("CONTRACTIVE FAILURE")
print("convex_set \n",convex_set)


starting vector [ 0.  0. -1.]
CONE SUCCESS ON LAST TRY
cone 
 [[-1.  0.  0.]
 [ 0. -2.  1.]
 [ 0.  1. -1.]
 [ 2.  0. -1.]]
CONTRACTIVE FAILURE
convex_set 
 [[-3.  0.  1.]
 [-2.  0.  0.]
 [-2.  2.  0.]
 [-1.  2.  0.]
 [ 0. -3.  2.]
 [ 0.  0. -1.]
 [ 0. -0.  1.]
 [ 0.  3. -2.]
 [ 1. -2.  0.]
 [ 2. -2.  0.]
 [ 2.  0.  0.]
 [ 3.  0. -1.]
 [ 3. -0. -1.]
 [ 2. -0. -0.]
 [ 2. -2. -0.]
 [ 1. -2. -0.]
 [-0.  3. -2.]
 [-0. -0.  1.]
 [-0.  0. -1.]
 [-0. -3.  2.]
 [-1.  2. -0.]
 [-2.  2. -0.]
 [-2. -0. -0.]
 [-3. -0.  1.]]
